In [1]:
import os

for i in range(5):
    url = f'https://f000.backblazeb2.com/file/malay-dataset/text-similarity/mnli/translated-{i}.json'
    os.system(f'wget {url}')

In [2]:
for i in range(7):
    url = f'https://f000.backblazeb2.com/file/malay-dataset/text-similarity/snli/part{i + 1}.json'
    os.system(f'wget {url}')

In [3]:
import re

def cleaning(string):
    string = string.replace('\n', ' ')
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

In [4]:
from glob import glob
import tensorflow as tf
import json
from pathlib import Path

labels = {'contradiction': 'percanggahan', 'entailment': 'berkait'}

files = glob('part*.json')

filename = 'snli.tsv'
with tf.io.gfile.GFile(filename, 'w') as outfile:
    for file in files:
        print(file)
        with open(file) as fopen:
            data = json.load(fopen)
        print(file, len(data))

        s = Path(file).stem
        for i in range(len(data)):
            if len(data[i]) != 2:
                continue

            label = labels.get(data[i][0], data[i][0])
            splitted = data[i][1].split(' <> ')
            if len(splitted) != 2:
                continue
            q = f'ayat1: {cleaning(splitted[0])} ayat2: {(splitted[1])}'
            outfile.write('%s\t%s\n' % (q, label))

part2.json
part2.json 49998
part7.json
part7.json 30934
part1.json
part1.json 50000
part4.json
part4.json 50000
part6.json
part6.json 100000
part3.json
part3.json 50000
part5.json
part5.json 49998


In [5]:
files = glob('translated-*.json')

filename = 'mnli.tsv'
with tf.io.gfile.GFile(filename, 'w') as outfile:
    for file in files:
        print(file)
        with open(file) as fopen:
            data = json.load(fopen)
        print(file, len(data))

        s = Path(file).stem
        for i in range(len(data)):
            if len(data[i]) != 3:
                continue

            label = labels.get(data[i][1], data[i][1])
            splitted = data[i][2].split(' <> ')
            if len(splitted) != 3:
                continue
            q = f'ayat1: {cleaning(splitted[0])} ayat2: {(splitted[1])}'
            outfile.write('%s\t%s\n' % (q, label))

translated-1.json
translated-1.json 99874
translated-4.json
translated-4.json 9994
translated-train.json
translated-train.json 96290
translated-3.json
translated-3.json 92581
translated-0.json
translated-0.json 99871
translated-validation.json
translated-validation.json 2338
translated-2.json
translated-2.json 99890


In [6]:
urls = [
'https://f000.backblazeb2.com/file/malay-dataset/text-similarity/quora/0-100k.json',
'https://f000.backblazeb2.com/file/malay-dataset/text-similarity/quora/100k-200k.json',
'https://f000.backblazeb2.com/file/malay-dataset/text-similarity/quora/200k-300k.json',
'https://f000.backblazeb2.com/file/malay-dataset/text-similarity/quora/300k-400k.json',
'https://f000.backblazeb2.com/file/malay-dataset/text-similarity/quora/400k-500k.json']

for url in urls:
    os.system(f'wget {url}')

In [7]:
files = glob('*0*-*k.json')
labels = {0: 'tak sama', 1: 'sama'}

filename = 'quora.tsv'
with tf.io.gfile.GFile(filename, 'w') as outfile:
    for file in files:
        with open(file) as fopen:
            data = json.load(fopen)
        print(file, len(data))
        s = Path(file).stem

        for i in range(len(data)):
            if len(data[i]) != 2:
                continue
            label = labels[data[i][1]]
            splitted = data[i][0].split(' <> ')
            if len(splitted) != 2:
                continue
            q = f'soalan1: {cleaning(splitted[0])} soalan2: {cleaning(splitted[1])}'
            outfile.write('%s\t%s\n' % (q, label))

0-100k.json 99993
400k-500k.json 4290
300k-400k.json 99995
200k-300k.json 99992
100k-200k.json 100000


In [8]:
import tensorflow as tf
import tensorflow_datasets as tfds
from t5.data import preprocessors as prep
import functools
import t5
import gin

gin.parse_config_file('pretrained_models_base_operative_config.gin')
vocab = 'sp10m.cased.t5.model'

In [14]:
def dataset(split, shuffle_files = False):
    del shuffle_files
    ds = tf.data.TextLineDataset(['quora.tsv', 'mnli.tsv', 'snli.tsv'])

    ds = ds.map(
        functools.partial(
            tf.io.decode_csv,
            record_defaults = ['', ''],
            field_delim = '\t',
            use_quote_delim = False,
        ),
        num_parallel_calls = tf.data.experimental.AUTOTUNE,
    )
    ds = ds.map(lambda *ex: dict(zip(['question', 'answer'], ex)))
    return ds

def preprocessor(ds):
    def to_inputs_and_targets(ex):
        return {'inputs': ex['question'], 'targets': ex['answer']}

    return ds.map(
        to_inputs_and_targets,
        num_parallel_calls = tf.data.experimental.AUTOTUNE,
    )

In [15]:
files = ['quora.tsv', 'mnli.tsv', 'snli.tsv']

t5.data.TaskRegistry.remove('similarity_dataset')
t5.data.TaskRegistry.add(
    'similarity_dataset',
    dataset_fn = dataset,
    splits = ['train'],
    text_preprocessor = [preprocessor],
    sentencepiece_model_path = vocab,
    metric_fns = [t5.evaluation.metrics.accuracy],
)

/home/husein/.local/lib/python3.6/site-packages/t5/models/mesh_transformer.py:210: UserWarning: get_sentencepiece_model_path is deprecated. Please pass the mixture or task vocabulary directly to the Mesh TensorFlow Transformer instead.
  "get_sentencepiece_model_path is deprecated. Please pass the mixture or "


In [16]:
nq_task = t5.data.TaskRegistry.get("similarity_dataset")
ds = nq_task.get_dataset(split=file, sequence_length={"inputs": 1024, "targets": 1024})

In [17]:
r = tfds.as_numpy(ds)

In [18]:
next(r)

{'inputs_plaintext': b'soalan1: Bagaimana saya boleh memuat turun kandungan dari torrent kickass tanpa pendaftaran? soalan2: Adakah Kickass Torrents boleh dipercayai?',
 'inputs': array([ 2105,   112,    50,  5268,   113,    66,  5870,  1340,  2097,
           37,    15,  1939, 12334, 22850,    13,    25,    25,   299,
         2372,   275,  2105,   143,    50,  3450,   313,  6269,    13,
           25,    25, 16063,  4355,    25,    66,  1306,   275,     1]),
 'targets_plaintext': b'tak sama',
 'targets': array([379,  96,   1])}